In [1]:

import numpy as np
import pandas as pd
from collections import Counter
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.snippets import sequence_padding
from bert4keras.snippets import uniout, open
from keras.models import Model
import tensorflow as tf
from tensorflow.python.platform import gfile
maxlen = 32




/home/zyp/anaconda3/envs/simbert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zyp/anaconda3/envs/simbert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zyp/anaconda3/envs/simbert/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zyp/anaconda3/envs/simbert/lib/python3.6/site-

In [2]:
dict_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/vocab.txt'

a = open(dict_path,"r")

a= a.read()

In [3]:
token_dict = {}
encoding='utf-8'
with open(dict_path, encoding=encoding) as reader:
    for line in reader:
        token = line.split()
        token = token[0] if token else line.strip()
        token_dict[token] = len(token_dict)

13685

In [26]:
len(a.split("\t"))

1

In [24]:
a.split("\n")[2232]

'屈'

In [4]:
# bert配置
config_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/home/zyp/pyprojects/simbert/kg/bert/chinese_simbert_L-12_H-768_A-12/vocab.txt'

token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)

# 建立分词器
#tokenizer = Tokenizer(dict_path, do_lower_case=True)  # 建立分词器

# 建立加载模型
# bert = build_transformer_model(
#     config_path,
#     checkpoint_path,
#     with_pool='linear',
#     application='unilm',
#     keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字表    
#     return_keras_model=False,
# )
# encoder = keras.models.Model(bert.model.inputs, bert.model.outputs[0])


In [5]:
sess = K.get_session()

In [4]:
graph = tf.graph_util.convert_variables_to_constants(sess,sess.graph_def,output_node_names=['Pooler-Dense/BiasAdd'])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 199 variables.
INFO:tensorflow:Converted 199 variables to const ops.


In [5]:
tf.train.write_graph(graph, "./", "test3.pb", False)

'./test3.pb'

In [2]:
sess = tf.Session()
with gfile.FastGFile("test3.pb", 'rb') as f:
    graph3 = tf.GraphDef()
    graph3.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph3,name="") # 导入计算图

Instructions for updating:
Use tf.gfile.GFile.


In [20]:
tf.global_variables()[::-1][5].op.op_def

[<tf.Variable 'MLM-Bias/bias:0' shape=(13584,) dtype=float32>,
 <tf.Variable 'MLM-Norm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'MLM-Norm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'MLM-Dense/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'MLM-Dense/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'Pooler-Dense/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'Pooler-Dense/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'Transformer-11-FeedForward-Norm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'Transformer-11-FeedForward-Norm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'Transformer-11-FeedForward/dense_72/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'Transformer-11-FeedForward/dense_72/kernel:0' shape=(3072, 768) dtype=float32>,
 <tf.Variable 'Transformer-11-FeedForward/dense_71/bias:0' shape=(3072,) dtype=float32>,
 <tf.Variable 'Transformer-11-FeedForward/dense_71/kernel:0' shape=(768, 3072) dtype=float32>,
 <tf.Variabl

In [3]:
sess.graph_def.node[-2]

NameError: name 'sess' is not defined

In [30]:
token1 = sequence_padding([[2, 2479, 617, 6826, 617, 6483, 3]],length=32)

In [31]:
np.zeros_like(token1)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [9]:
rexcel = pd.read_excel("./datasets/爱善大学堂百问百答模块.xlsx",sheet_name='问答明细')
rexcel["Unnamed: 1"][2:]
asks = rexcel["Unnamed: 1"][2:].apply(lambda x:tokenizer.encode(x, max_length=maxlen)[0]).tolist()
asks = sequence_padding(asks)

In [24]:
texts = rexcel["Unnamed: 1"][2:].tolist()

In [16]:
vecs = asks_vecs

In [11]:
asks_vecs = sess.run("Pooler-Dense/BiasAdd:0",feed_dict={"Input-Token:0":asks,"Input-Segment:0":np.zeros_like(asks)})

In [13]:
asks_vecs /= (asks_vecs**2).sum(axis=1,keepdims=True)**0.5

In [14]:
asks_vecs = asks_vecs.reshape(-1,768)

In [25]:
def most_similar(text, topn=10):
    """检索最相近的topn个句子
    """
    token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
    vec = sess.run("Pooler-Dense/BiasAdd:0",feed_dict={"Input-Token:0":[token_ids],"Input-Segment:0":[segment_ids]})[0]
    vec /= (vec**2).sum()**0.5
    sims = np.dot(vecs, vec)
    return [(texts[i], sims[i]) for i in sims.argsort()[::-1][:topn]]

In [26]:
most_similar("怎么跟客户介绍俏妃")

[('如何引导俏妃的好评反馈？', 0.8197763),
 ('怎么给年轻女顾客连带俏妃送给妈妈？', 0.753228),
 ('俏妃发快递时如何给客户发信息？', 0.751461),
 ('如何对俏妃客户进行半个月后的连带回访？', 0.7250136),
 ('怎么给老顾客连带俏妃青春版送女儿？', 0.69393027),
 ('客户说：俏妃卫生巾有点贵？', 0.6606858),
 ('俏妃经典款（老款）的核心卖点是什么？', 0.6295761),
 ('客户说：我用棉柔卫生巾、网面卫生巾挺好的，不需要俏妃？', 0.5946022),
 ('俏妃青春版（少女）的核心卖点是什么？', 0.5549495),
 ('俏妃青春版如何做到超强防漏？', 0.554675)]

In [4]:
sess.graph.get_operations()[::-1]

[<tf.Operation 'Pooler-Dense/BiasAdd' type=BiasAdd>,
 <tf.Operation 'Pooler-Dense/BiasAdd/ReadVariableOp' type=Identity>,
 <tf.Operation 'Pooler-Dense/MatMul' type=MatMul>,
 <tf.Operation 'Pooler-Dense/MatMul/ReadVariableOp' type=Identity>,
 <tf.Operation 'Pooler-Dense/bias' type=Const>,
 <tf.Operation 'Pooler-Dense/kernel' type=Const>,
 <tf.Operation 'Pooler/strided_slice' type=StridedSlice>,
 <tf.Operation 'Pooler/strided_slice/stack_2' type=Const>,
 <tf.Operation 'Pooler/strided_slice/stack_1' type=Const>,
 <tf.Operation 'Pooler/strided_slice/stack' type=Const>,
 <tf.Operation 'Transformer-11-FeedForward-Norm/add_1' type=Add>,
 <tf.Operation 'Transformer-11-FeedForward-Norm/add_1/ReadVariableOp' type=Identity>,
 <tf.Operation 'Transformer-11-FeedForward-Norm/mul' type=Mul>,
 <tf.Operation 'Transformer-11-FeedForward-Norm/mul/ReadVariableOp' type=Identity>,
 <tf.Operation 'Transformer-11-FeedForward-Norm/truediv' type=RealDiv>,
 <tf.Operation 'Transformer-11-FeedForward-Norm/Sqrt' ty

In [5]:
sess.graph.get_tensor_by_name("Input-Token:0")

<tf.Tensor 'Input-Token:0' shape=(?, ?) dtype=float32>

In [27]:
import requests

### 模型压缩

In [2]:
converter = tf.contrib.lite.TocoConverter.from_saved_model("test3.pb")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



AttributeError: module 'tensorflow.contrib' has no attribute 'lite'

In [3]:
dir( tf.contrib)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'autograph',
 'batching',
 'bayesflow',
 'bigtable',
 'checkpoint',
 'cloud',
 'cluster_resolver',
 'compiler',
 'constrained_optimization',
 'copy_graph',
 'crf',
 'cudnn_rnn',
 'data',
 'decision_trees',
 'deprecated',
 'distribute',
 'distributions',
 'eager',
 'estimator',
 'factorization',
 'feature_column',
 'ffmpeg',
 'framework',
 'gan',
 'graph_editor',
 'grid_rnn',
 'image',
 'input_pipeline',
 'integrate',
 'keras',
 'kernel_methods',
 'labeled_tensor',
 'layers',
 'learn',
 'legacy_seq2seq',
 'linear_optimizer',
 'lookup',
 'losses',
 'memory_stats',
 'meta_graph_transform',
 'metrics',
 'mixed_precision',
 'model_pruning',
 'nn',
 'opt',
 'optimizer_v2',
 'periodic_resample',
 'predictor',
 'proto',
 'quantization',
 'quantize',
 'receptive_field',
 'recurrent',
 'reduce_slice_ops',
 'remote_fused_graph',
 'resampler',
 'rnn',
 'rpc',
 'saved_mode